In [1]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [11]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageEnhance, ImageOps, ImageTk, ImageDraw

class ImageEditorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Editor")
        self.root.geometry("800x600")
        self.root.configure(bg="#2c3e50")

        self.image = None
        self.original_image = None
        self.cropping = False
        self.crop_start_x = 0
        self.crop_start_y = 0
        self.crop_end_x = 0
        self.crop_end_y = 0
        self.crop_rectangle = None

        # Styling options
        self.button_style = {"font": ("Arial", 12), "bg": "#3498db", "fg": "white", "activebackground": "#2980b9", "activeforeground": "white", "bd": 0, "relief": "flat"}

        # Buttons
        self.load_button = tk.Button(root, text="Load Image", command=self.load_image, **self.button_style)
        self.load_button.pack(pady=10)

        self.brightness_label = tk.Label(root, text="Brightness", bg="#2c3e50", fg="white", font=("Arial", 12))
        self.brightness_label.pack()
        self.brightness_slider = tk.Scale(root, from_=0.5, to=2.0, orient=tk.HORIZONTAL, resolution=0.1, bg="#2c3e50", fg="white", command=self.adjust_brightness)
        self.brightness_slider.set(1.0)
        self.brightness_slider.pack(pady=5)

        self.contrast_label = tk.Label(root, text="Contrast", bg="#2c3e50", fg="white", font=("Arial", 12))
        self.contrast_label.pack()
        self.contrast_slider = tk.Scale(root, from_=0.5, to=2.0, orient=tk.HORIZONTAL, resolution=0.1, bg="#2c3e50", fg="white", command=self.adjust_contrast)
        self.contrast_slider.set(1.0)
        self.contrast_slider.pack(pady=5)

        self.warmth_label = tk.Label(root, text="Warmth", bg="#2c3e50", fg="white", font=("Arial", 12))
        self.warmth_label.pack()
        self.warmth_slider = tk.Scale(root, from_=0.5, to=2.0, orient=tk.HORIZONTAL, resolution=0.1, bg="#2c3e50", fg="white", command=self.adjust_warmth)
        self.warmth_slider.set(1.0)
        self.warmth_slider.pack(pady=5)

        self.crop_button = tk.Button(root, text="Start Cropping", command=self.start_cropping, **self.button_style)
        self.crop_button.pack(pady=10)

        self.black_white_button = tk.Button(root, text="Black & White", command=self.convert_black_white, **self.button_style)
        self.black_white_button.pack(pady=10)

        self.reset_button = tk.Button(root, text="Reset", command=self.reset_image, **self.button_style)
        self.reset_button.pack(pady=10)

        self.save_button = tk.Button(root, text="Save Image", command=self.save_image, **self.button_style)
        self.save_button.pack(pady=10)

        self.image_label = tk.Label(root, bg="#2c3e50")
        self.image_label.pack()

        self.image_canvas = tk.Canvas(root, bg="#2c3e50", highlightthickness=0)
        self.image_canvas.pack()

        self.root.bind("<ButtonPress-1>", self.on_mouse_down)
        self.root.bind("<B1-Motion>", self.on_mouse_move)
        self.root.bind("<ButtonRelease-1>", self.on_mouse_up)

    def load_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.image = Image.open(file_path)
            self.image_path = file_path
            self.original_image = self.image.copy()
            self.display_image(self.image)
            messagebox.showinfo("Image Editor", "Image Loaded Successfully")

    def display_image(self, img):
        self.image_canvas.delete("all")
        img.thumbnail((400, 400))
        self.img_tk = ImageTk.PhotoImage(img)
        self.image_canvas.config(width=self.img_tk.width(), height=self.img_tk.height())
        self.image_canvas.create_image(0, 0, anchor=tk.NW, image=self.img_tk)

    def start_cropping(self):
        self.cropping = True

    def on_mouse_down(self, event):
        if self.cropping and self.image:
            self.crop_start_x = event.x
            self.crop_start_y = event.y
            self.crop_rectangle = self.image_canvas.create_rectangle(self.crop_start_x, self.crop_start_y, self.crop_start_x, self.crop_start_y, outline="red")

    def on_mouse_move(self, event):
        if self.cropping and self.image and self.crop_rectangle:
            self.crop_end_x = event.x
            self.crop_end_y = event.y
            self.image_canvas.coords(self.crop_rectangle, self.crop_start_x, self.crop_start_y, self.crop_end_x, self.crop_end_y)

    def on_mouse_up(self, event):
        if self.cropping and self.image and self.crop_rectangle:
            self.crop_end_x = event.x
            self.crop_end_y = event.y
            self.crop_image()
            self.cropping = False
            self.image_canvas.delete(self.crop_rectangle)
            self.crop_rectangle = None

    def crop_image(self):
        if self.image:
            crop_box = (self.crop_start_x, self.crop_start_y, self.crop_end_x, self.crop_end_y)
            self.image = self.image.crop(crop_box)
            self.original_image = self.image.copy()
            self.display_image(self.image)
            messagebox.showinfo("Image Editor", "Image Cropped Successfully")

    def adjust_brightness(self, value):
        if self.image:
            try:
                enhancer = ImageEnhance.Brightness(self.original_image)
                self.image = enhancer.enhance(float(value))
                self.display_image(self.image)
            except Exception as e:
                messagebox.showerror("Error", f"Failed to adjust brightness: {e}")

    def adjust_contrast(self, value):
        if self.image:
            try:
                enhancer = ImageEnhance.Contrast(self.original_image)
                self.image = enhancer.enhance(float(value))
                self.display_image(self.image)
            except Exception as e:
                messagebox.showerror("Error", f"Failed to adjust contrast: {e}")

    def adjust_warmth(self, value):
        if self.image:
            try:
                r, g, b = self.original_image.split()
                r = r.point(lambda p: p * float(value))
                self.image = Image.merge("RGB", (r, g, b))
                self.display_image(self.image)
            except Exception as e:
                messagebox.showerror("Error", f"Failed to adjust warmth: {e}")

    def convert_black_white(self):
        if self.image:
            try:
                self.image = ImageOps.grayscale(self.image)
                self.display_image(self.image)
                messagebox.showinfo("Image Editor", "Converted to Black & White Successfully")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to convert to black & white: {e}")

    def reset_image(self):
        if self.image:
            self.image = self.original_image.copy()
            self.display_image(self.image)
            self.brightness_slider.set(1.0)
            self.contrast_slider.set(1.0)
            self.warmth_slider.set(1.0)
            messagebox.showinfo("Image Editor", "Image Reset Successfully")
        else:
            messagebox.showerror("Error", "No image loaded")

    def save_image(self):
        if self.image:
            file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"), ("All files", "*.*")])
            if file_path:
                self.image.save(file_path)
                messagebox.showinfo("Image Editor", "Image Saved Successfully")

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageEditorApp(root)
    root.mainloop()


